In [1]:
# Define data
α_1 = 0.4009298
γ_1 = 0.464835
R = 6378*1000. #m 
η_A = 0.29
Q = 1367. #W/m^2
EN = 40
D_r = 5.3 #m
L = 9.777 
k = 1.38064852e-23 #J/K Boltzman constant R/Na
T_s = 135. #K
B = 8.
N = 2000. # pixel width
η = 0.55
λ_v = 500e-9 #nm
f = 2.2e9 #Hz
c = 3e8  #m/s
λ_c = c/f
G_r = η*(π*D_r/λ_c)^2
ρ_A = 10. #kg/m^2
ρ_p = 100. #kg/m^1.5
ρ_T = 2. #kg/m^1.5
ρ_P = 500.*1000 #kg*km
ρ_b = 0.0001 #kg/J
P_l = 5 #W
μ = 3.986005e14 # m^3/s^2 standard gravitational parameter
X_r = 20. #m/px
m_c = 0.2 #kg, some required weight
η_S = 0.2 # 20% of total mass is structural mass
;

In [2]:
sD_p = 0.025903142126747384
sD_T = 0.45294154570683576
sA = 0.023702138567413843
sm_t = 4.878403428287764
sm_b = 1.95657129093823
sm_A = 0.23702136363256185
sm_p = 0.41689666235172684
sm_T = 0.60966771484270366
sm_S = 0.97568107830481066 # structural mass
sm_P = 0.48256714850501831
sP_t = 9.3962436715381017
sP_T = 1.3167071017983942 
sE_b = 19565.713457688278
sh = 1036124.9254905024
sa = 7414226.9926438555
sT = 6353.4519745674825
sr = 3780286.1998746251
sd = 0.67225899734205319
se = 0.32774185759103097
sg = 0.17225970984913666;

In [3]:
sP_t, sA*η_A*Q 

(9.396243671538102, 9.396238792279869)

In [4]:
sP_T+P_l,sd*sP_t

(6.316707101798395, 6.316709349409817)

In [5]:
sE_b,sP_t*se*sT

(19565.713457688278, 19565.72445796789)

In [6]:
EN*L*k*T_s*R*2*π*N*B*4*sr^2, sP_T*G_r*X_r*sg*sT*η*sD_T^2

(2.671629471818256e7, 2.6667153937523004e7)

In [7]:
X_r*sD_p, sh*λ_v

(0.5180628425349477, 0.5180624627452511)

In [8]:
sa, R + sh

(7.4142269926438555e6, 7.414124925490502e6)

In [9]:
sg, α_1*sh^γ_1*R^(-γ_1)

(0.17225970984913666, 0.17226097631835688)

In [10]:
sT, 2*π*sa^(3/2)*μ^(-1/2)

(6353.4519745674825, 6353.452188119963)

In [11]:
sr^2, sh^2 + 2*R*sh 

(1.4290563752962533e13, 1.4290364410779547e13)

In [12]:
(sm_b,ρ_b*sE_b), (sm_p,ρ_p*sD_p^1.5), (sm_A,ρ_A*sA),
(sm_T,ρ_T*sD_T^1.5), (sm_P,ρ_P/sh), (sm_S,η_S*sm_t),
(sm_t,sm_b+sm_p+sm_A+sm_T +sm_P +sm_S)

((1.95657129093823, 1.9565713457688279), (0.41689666235172684, 0.41689690664827783), (0.23702136363256185, 0.23702138567413844), (0.6096677148427037, 0.6096677639575033), (0.4825671485050183, 0.482567292513786), (0.9756810783048107, 0.9756806856575528), (4.878403428287764, 4.678405258575052))

In [13]:
using Ipopt
solver = IpoptSolver(print_level=0)
method = :LogSumExp;

In [14]:
using JuMP
using JuGP

In [15]:
using ECOS
solver = ECOSSolver(verbose=true)
method = :Conic;

In [18]:
# Define JuMP model
m = GPModel(method=method, solver=solver)

@variables m begin
    0 <= D_p <= 0.5 # payload optical aperture diameter
    0 <= D_T <= 1.0 # transmitter antenna diameter
    0 <= A <= 0.1 # solar panel surface area

    # Mass related parameters
    0 <= m_t <= 10. # total
    0 <= m_b <= 10. # battery mass
    0 <= m_A <= 10. # solar panel mass
    0 <= m_p <= 10. # payload mass
    0 <= m_T <= 10. # transmitter mass
    0 <= m_S <= 10. # structural mass
    0 <= m_P <= 10. # propulsion mass

    # Power related parameters
    0 <= P_t <= 100. # total power
    0 <= P_T <= 100. # transmitter power
    0 <= E_b <= 100000. # battery energy

    # Orbit related parameters
    100000. <= h <= 2000000. # altitude
    R+100000. <= a <= 2000000.+R # semi-major axis
    5000. <= T <= 10000. # orbit period 
    100000. <= r <= 5000000. # worst case communications distance
    0 <= d <= 1. # daylight fraction of orbit
    0 <= e <= 1. # eclipse faction of orbit
    0 <= g <= 10000. # ground station viewing fraction of orbit
end

# TODO some of these may be better defined as NLexpressions
@NLconstraints m begin
    # Power and communications
    P_t == A*η_A*Q # all power generated by solar panel
    P_T + P_l <= d*P_t
    E_b >= P_t*e*T
    EN*L*k*T_s*R*2*π*N*B*4*r^2 == P_T*G_r*X_r*g*T*η*D_T^2

    # Payload performance
    X_r*D_p == h*λ_v

    # Orbit
#     e >= 1 - d
#     d <= g + 0.5
    a >= R + h
    g == α_1*h^γ_1*R^(-γ_1)
    T == 2*π*a^(3/2)*μ^(-1/2)
    r^2 >= h^2 + 2*R*h 

    # Mass budgets
    m_b == ρ_b*E_b
    m_p == ρ_p*D_p^1.5
    m_A == ρ_A*A
    m_T == ρ_T*D_T^1.5
    m_P == ρ_P/h
    m_S == η_S*m_t
    m_t >= m_b + m_p + m_A + m_T + m_P + m_S
end

@NLobjective(m, Min, m_t)
;

In [19]:
solve(m);
getobjectivevalue(m)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{DataStructures.OrderedDict{UInt64,Tuple{Union{AbstractArray, Number},Union{AbstractArray, Number}}}}, ::Int64) at .\deprecated.jl:57
 [3] conic_form!(::Convex.ExpConstraint, ::Convex.UniqueConicForms) at C:\Users\johan\.julia\v0.6\Convex\src\constraints\exp_constraints.jl:52
 [4] conic_form!(::Convex.ExpAtom, ::Convex.UniqueConicForms) at C:\Users\johan\.julia\v0.6\Convex\src\atoms/exp_cone\exp.jl:55
 [5] conic_form!(::Convex.SumAtom, ::Convex.UniqueConicForms) at C:\Users\johan\.julia\v0.6\Convex\src\atoms/affine\sum.jl:48
 [6] conic_form!(::Convex.LogSumExpAtom, ::Convex.UniqueConicForms) at C:\Users\johan\.julia\v0.6\Convex\src\atoms/exp_cone\logsumexp.jl:55
 [7] conic_form!(::Convex.NegateAtom, ::Convex.UniqueConicForms) at C:\Users\johan\.julia\v0.6\Convex\src\atoms/affine\add_subtract.jl:47
 [8] conic_form!(::Convex.AdditionAtom, ::Convex.UniqueConicForms) at C:\Users\johan\.julia\v0.6\Convex\src

0


ECOS 2.0.2 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -6.357e+01  +7e+01  1e+00  9e-01  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  +5.009e-01  -4.663e+01  +3e+01  7e-01  6e-01  1e+00  4e-01  0.6253  8e-02   1  1  1 |  0  0
 2  +5.712e-01  -4.968e+01  +2e+01  7e-01  3e-01  2e+00  3e-01  0.6463  5e-01   1  1  1 |  0  0
 3  +7.705e-01  -4.408e+01  +6e+00  8e-01  2e-01  7e+00  9e-02  0.9791  3e-01   1  1  1 |  1  0
 4  +8.601e-01  -2.377e+01  +2e+00  7e-01  2e-01  3e+01  2e-02  0.7517  3e-03   1  1  1 |  0  1
 5  +9.839e-01  +6.378e+02  +5e-02  7e-01  2e-01  7e+02  8e-04  0.9778  1e-02   2  1  1 |  1  0
 6  +9.395e-01  +8.926e+03  +4e-03  7e-01  2e-01  9e+03  6e-05  0.9418  2e-02   3  2  1 |  1  0
 7  +9.282e-01  +3.713e+04  +1e-03  7e-01  1e-01  4e+04  1e-05  0.7755  2e-02   1  2  1 |  1  1
 8  +9.277e-01  +1.772e+06  +2e-05  7e-01  1e-


Constraint expr: x[11] - x[3] * 0.29 * 1367.0 == 0.0
Constraint type: ==
[0.0025225134323840276*x_{3}^{-1.0}*x_{11}^{1.0} + -1.0]

Constraint expr: (x[12] + 5.0) - x[18] * x[11] <= 0.0
Constraint type: <=
[1.0*x_{11}^{-1.0}*x_{12}^{1.0}*x_{18}^{-1.0} + 5.0*x_{11}^{-1.0}*x_{18}^{-1.0} + -1.0]

Constraint expr: x[13] - x[11] * x[19] * x[16] >= 0.0
Constraint type: >=
[1.0*x_{13}^{-1.0}*x_{19}^{1.0}*x_{16}^{1.0}*x_{11}^{1.0} + -1.0]

Constraint expr: 40.0 * 9.777 * 1.38064852e-23 * 135.0 * 6.378e6 * 2.0 * 3.141592653589793 * 2000.0 * 8.0 * 4.0 * x[17] ^ 2.0 - x[12] * 2610.1601421768432 * 20.0 * x[20] * x[16] * 0.55 * x[2] ^ 2.0 == 0.0
Constraint type: ==
[6.511289802055442e-11*x_{2}^{-2.0}*x_{16}^{-1.0}*x_{17}^{2.0}*x_{12}^{-1.0}*x_{20}^{-1.0} + -1.0]

Constraint expr: 20.0 * x[1] - x[14] * 5.0e-7 == 0.0
Constraint type: ==
[4.0e7*x_{14}^{-1.0}*x_{1}^{1.0} + -1.0]

Constraint expr: x[15] - (6.378e6 + x[14]) >= 0.0
Constraint type: >=
[6.378e6*x_{15}^{-1.0} + 1.0*x_{14}^{1.0}*x_{15}^{-1.0

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{DataStructures.OrderedDict{UInt64,Tuple{Union{AbstractArray, Number},Union{AbstractArray, Number}}}}, ::Int64) at ./deprecated.jl:57
 [3] conic_form!(::Convex.ExpConstraint, ::Convex.UniqueConicForms) at /home/coey/.julia/v0.6/Convex/src/constraints/exp_constraints.jl:52
 [4] conic_form!(::Convex.ExpAtom, ::Convex.UniqueConicForms) at /home/coey/.julia/v0.6/Convex/src/atoms/exp_cone/exp.jl:55
 [5] conic_form!(::Convex.SumAtom, ::Convex.UniqueConicForms) at /home/coey/.julia/v0.6/Convex/src/atoms/affine/sum.jl:48
 [6] conic_form!(::Convex.LogSumExpAtom, ::Convex.UniqueConicForms) at /home/coey/.julia/v0.6/Convex/src/atoms/exp_cone/logsumexp.jl:55
 [7] conic_form!(::Convex.NegateAtom, ::Convex.UniqueConicForms) at /home/coey/.julia/v0.6/Convex/src/atoms/affine/add_subtract.jl:47
 [8] conic_form!(::Convex.AdditionAtom, ::Convex.UniqueConicForms) at /home/coey/.julia/v0.6/Convex/src/atoms/affine/add_subtra


ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -1.182e+02  +7e+01  1e+00  6e-01  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  +1.735e+00  -9.042e+01  +3e+01  7e-01  5e-01  2e+00  4e-01  0.7091  1e-01   1  1  1 |  0  0
 2  +3.849e+00  -5.666e+01  +1e+01  4e-01  4e-01  2e+00  2e-01  0.5668  6e-02   1  1  1 |  0  2
 3  +6.000e+00  -1.935e+01  +4e+00  2e-01  2e-01  6e-01  6e-02  0.7058  1e-01   1  1  1 |  1  1
 4  +7.133e+00  +6.630e-01  +1e+00  4e-02  5e-02  1e-01  2e-02  0.7833  6e-02   1  1  1 |  1  1
 5  +7.367e+00  +5.417e+00  +4e-01  1e-02  2e-02  4e-02  6e-03  0.6947  4e-02   1  1  1 |  1  1
 6  +7.434e+00  +6.935e+00  +1e-01  3e-03  6e-03  1e-02  2e-03  0.7438  2e-02   1  1  1 |  1  1
 7  +7.451e+00  +7.311e+00  +3e-02  8e-04  2e-03  3e-03  5e-04  0.7285  3e-02   1  1  1 |  1  1
 8  +7.456e+00  +7.417e+00  +9e-03  2e-04  7e-

:Optimal